# SST Empirical Orthogonal Function Analysis

This notebook will import SST data from a source, select it to be inside the scope of the project and do EOF analysis to determine Central Atlantic Niño Index and Eastern Atlantic Niño Index.

# Imports

In [2]:
import numpy as np
import pandas as pd
import xarray as xr
import dask
from dask_jobqueue import PBSCluster
from dask.distributed import Client
import xeofs as xe
import glob
from geocat.viz import util as gvutil
import cartopy.crs as ccrs
import cartopy.feature as cf
import cartopy.util as cutil
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import colormaps as cm
import matplotlib
import matplotlib.pyplot as plt

# PBSCluster

In [13]:
# # Create a PBS cluster object
# cluster = PBSCluster(account='P93300313',
#                      job_name='CANI_EANI_EOFa',
#                      cores=1,
#                      memory='8GiB',
#                      processes=1,
#                      walltime='02:00:00',
#                      queue='casper',
#                      interface='ext',
#                      n_workers=1)

# # dont scale many workers unless using LE
# # cluster.scale(10)

# client = Client(cluster)
# client

/glade/u/home/acruz/.conda/envs/EOFa_2025/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 35845 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/acruz/proxy/35845/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/acruz/proxy/35845/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://128.117.208.214:37367,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/acruz/proxy/35845/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [15]:
# cluster.scale(2)

In [16]:
# client.shutdown()
# cluster.workers

{'PBSCluster-0': <dask_jobqueue.pbs.PBSJob: status=running>,
 'PBSCluster-1': <dask_jobqueue.pbs.PBSJob: status=running>}

# Useful Functions

In [2]:
def ds_map(ds_to_plt, bounds=[20, -60, 10, -10], name='figure'):
    fig, ax = plt.subplots(1, 1,
                           subplot_kw={'projection': ccrs.PlateCarree()})
    fig.subplots_adjust(hspace=0, wspace=0, top=0.925, left=0.1)
    cbar_ax = fig.add_axes([0, 0, 0.1, 0.1])
    cdat, clon = cutil.add_cyclic_point(ds_to_plt, ds_to_plt.longitude)

    ax.set_title(name)
    lat_ticks = np.arange(bounds[3], bounds[2], 5)
    lon_ticks = np.arange(bounds[1], bounds[0], 10)
    ax.set_xticks(lon_ticks, crs=ccrs.PlateCarree())
    ax.set_yticks(lat_ticks, crs=ccrs.PlateCarree())
    lon_formatter = LongitudeFormatter(zero_direction_label=True)
    lat_formatter = LatitudeFormatter()
    ax.xaxis.set_major_formatter(lon_formatter)
    ax.yaxis.set_major_formatter(lat_formatter)
    ax.add_feature(cf.LAND)


    def resize_colobar(event):
        plt.draw()
        posn = ax.get_position()
        cbar_ax.set_position([posn.x0 + posn.width + 0.01, posn.y0,
                              0.04, posn.height])

    ax.set_extent(bounds, ccrs.PlateCarree())
    sst_contour = ax.contourf(clon, ds_to_plt.latitude, cdat,
                              levels=np.arange(-0.4, 0.5, 0.05),
                              # levels=40,
                              transform=ccrs.PlateCarree(), cmap='inferno', extend='both')
    fig.canvas.mpl_connect('resize_event', resize_colobar)
    ax.coastlines()
    plt.colorbar(sst_contour, cax=cbar_ax)
    resize_colobar(None)
    # plt.savefig(name, dpi=300)
    plt.show()


def detrend_dim(da, dim, deg=1):
    # detrend along a single dimension
    p = da.polyfit(dim=dim, deg=deg)
    fit = xr.polyval(da[dim], p.polyfit_coefficients)
    return da - fit


def index_plot(ds1, name1='', threshold=0.5):
    lim = 4 * threshold
    fig, ax = plt.subplots(figsize=(12, 6))

    ax.plot(ds1.time, ds1, color='black', label=name1)
    gvutil.add_major_minor_ticks(ax, x_minor_per_major=15, y_minor_per_major=3,
                                 labelsize=20)

    gvutil.set_axes_limits_and_ticks(ax, ylim=(-1*lim, lim))
    ax.fill_between(ds1.time, ds1, y2=-threshold,
                    where=ds1 < -threshold, color='blue', interpolate=True)
    ax.fill_between(ds1.time, ds1, y2=threshold,
                    where=ds1 > threshold, color='red', interpolate=True)
    plt.title(f'{name1}')
    ax.set_xlabel('year', fontsize=24)
    plt.grid()
    plt.show()

# Data import

In [9]:
def le_in(member_names, decades):
    data = []
    members = []
    for m in members:
        for d in decades:
            file = f'{path}b.e21.BHISTcmip6.f09_g17.LE2-{m}.cam.h0.SST.188001-188912.nc'
            d_data = xr.open_dataset(file)
            data.append(d_data)
        member_data = xr.concat(data, dim='time')
        members.append(member_data)
    xr.concat(members, dim='time')
    return members


# start.member_n
# bhist
# dates in scope 
hist_decades = ['191001-191912', '192001-192912',
                '193001-193912', '194001-194912', '195001-195912', '196001-196912',
                '197001-197912', '198001-198912', '199001-199912', '200001-200912', 
                '201001-201412']
# too many different member names
member_names = ['1001.001', '1021.002', '1041.003', '1061.004', '1081.005',
                '1101.006', '1121.007', '1141.008', '1161.009', '1181.010']


path = "/glade/campaign/cgd/cesm/CESM2-LE/timeseries/atm/proc/tseries/month_1/SST/"
hist = "b.e21.BHIST*"
type = '.nc'
files = path + hist + member_names[0]+ '*' + hist_decades[0] + type
hist_names = glob.glob(files)
hist_names

['/glade/campaign/cgd/cesm/CESM2-LE/timeseries/atm/proc/tseries/month_1/SST/b.e21.BHISTcmip6.f09_g17.LE2-1001.001.cam.h0.SST.191001-191912.nc']

In [19]:
# CESM SST
# files = glob.glob('/glade/campaign/cgd/cesm/CESM2-LE/timeseries/atm/proc/tseries/month_1/SST/*.nc')
files = ["/glade/campaign/cgd/cesm/CESM2-LE/timeseries/atm/proc/tseries/month_1/SST/b.e21.BHISTcmip6.f09_g17.LE2-1231.006.cam.h0.SST.188001-188912.nc",
         "/glade/campaign/cgd/cesm/CESM2-LE/timeseries/atm/proc/tseries/month_1/SST/b.e21.BHISTcmip6.f09_g17.LE2-1231.006.cam.h0.SST.189001-189912.nc"]
ds = xr.open_mfdataset(files, parallel=True)
ds

<xarray.Dataset> Size: 54MB
Dimensions:       (time: 240, lat: 192, zlon: 1, nbnd: 2, lev: 32, ilev: 33,
                   lon: 288)
Coordinates:
  * lat           (lat) float64 2kB -90.0 -89.06 -88.12 ... 88.12 89.06 90.0
  * lon           (lon) float64 2kB 0.0 1.25 2.5 3.75 ... 356.2 357.5 358.8
  * zlon          (zlon) float64 8B 0.0
  * lev           (lev) float64 256B 3.643 7.595 14.36 ... 957.5 976.3 992.6
  * ilev          (ilev) float64 264B 2.255 5.032 10.16 ... 967.5 985.1 1e+03
  * time          (time) object 2kB 1880-02-01 00:00:00 ... 1900-01-01 00:00:00
Dimensions without coordinates: nbnd
Data variables: (12/27)
    gw            (time, lat) float64 369kB dask.array<chunksize=(120, 192), meta=np.ndarray>
    zlon_bnds     (time, zlon, nbnd) float64 4kB dask.array<chunksize=(120, 1, 2), meta=np.ndarray>
    hyam          (time, lev) float64 61kB dask.array<chunksize=(120, 32), meta=np.ndarray>
    hybm          (time, lev) float64 61kB dask.array<chunksize=(120, 32), meta=np.ndarray>
    P0            (time) float64 2kB 1e+05 1e+05 1e+05 ... 1e+05 1e+05 1e+05
    hyai          (time, ilev) float64 63kB dask.array<chunksize=(120, 33), meta=np.ndarray>
    ...            ...
    n2ovmr        (time) float64 2kB dask.array<chunksize=(120,), meta=np.ndarray>
    f11vmr        (time) float64 2kB dask.array<chunksize=(120,), meta=np.ndarray>
    f12vmr        (time) float64 2kB dask.array<chunksize=(120,), meta=np.ndarray>
    sol_tsi       (time) float64 2kB dask.array<chunksize=(120,), meta=np.ndarray>
    nsteph        (time) float64 2kB dask.array<chunksize=(120,), meta=np.ndarray>
    SST           (time, lat, lon) float32 53MB dask.array<chunksize=(1, 192, 288), meta=np.ndarray>
Attributes:
    Conventions:       CF-1.0
    source:            CAM
    case:              b.e21.BHISTcmip6.f09_g17.LE2-1231.006
    logname:           sunseon
    host:              mom2
    initial_file:      b.e21.B1850.f09_g17.CMIP6-piControl.001.cam.i.1231-01-...
    topography_file:   /mnt/lustre/share/CESM/cesm_input/atm/cam/topo/fv_0.9x...
    model_doi_url:     https://doi.org/10.5065/D67H1H0V
    time_period_freq:  month_1

# Select data

In [20]:
# dates not necessary since we are doing all data
# ATL area
ATL_hadisst = ds['SST'].sel(lat=slice(10, -10), lon=slice(-60, 20)).compute()

# Anomalies

In [21]:
# the mistake was missing the groupby function
# all year
ATL_clim = ATL_hadisst.groupby(ATL_hadisst['time'].dt.month).mean(dim='time').compute()
ATL_anom_pm = ATL_hadisst.groupby(ATL_hadisst['time'].dt.month) - ATL_clim

# all anomaly fields were linearly detrended
ATL_anom_dtrend = detrend_dim(ATL_anom_pm, dim='time')

ValueError: cannot reshape array of size 0 into shape (0,17,newaxis)

# EOF

In [ ]:
model = xe.single.EOF(n_modes=3, use_coslat=True)
# all year climatologies of all data
model.fit(ATL_anom_dtrend, dim='time')
# # remove detrend
# model.fit(ATL_anom_pm, dim='time')
# components = model.components()
xplained_var = model.explained_variance_ratio().values

# PC and EOF normalizing and scaling

In [ ]:
# scale by PC std
# nomalized in this package is L2 norm not STD
PCs = model.scores(normalized=False)

# normalized by l2norm true as test
# PCs = model.scores()

pc_std = PCs.std()
pc_mean = PCs.mean()

normalized_PCs = (PCs - pc_mean) / pc_std
scaled_EOF = components * pc_std

In [ ]:
PC1 = normalized_PCs.sel(mode=1)
PC2 = normalized_PCs.sel(mode=2)
PC3 = normalized_PCs.sel(mode=3)

# EAN and CAN combined EOF patterns

In [ ]:
EATLs = (scaled_EOF.sel(mode=1) + scaled_EOF.sel(mode=3)) / (2 ** 0.5)
CATLs = (scaled_EOF.sel(mode=1) - scaled_EOF.sel(mode=3)) / (2 ** 0.5)

# CANI and EANI

In [ ]:
EANI = (PC1 + PC3) / (2 ** 0.5)
CANI = (PC1 - PC3) / (2 ** 0.5)

# using 5 months, find literature to change this
roll = 5
EANI_roll = EANI.rolling(time=roll, center=True).mean()
CANI_roll = CANI.rolling(time=roll, center=True).mean()

# Variability

In [ ]:
'''
The very last plot starts in 1990, but for a 21-year rolling window,
the data should start in 1980. So you may have a bug somewhere.

For the variance plots, we should use JJA mean EANI and CANI. 
Before rolling, you should calculate the 3 month-mean and group them by year.
For example:

EANI_JJA = EANI.where(EANId.time.dt.month.isin([7,8,9]),drop=True)
EANI_JJA_mean = EANI_JJA.groupby(EANI_JJA.time.dt.year).mean()
EANI_roll = EANI_JJA_mean.rolling(year=21, center=True).mean()
'''
# select only the summer of the index
EANI_JJA = EANI_roll.where(EANI_roll.time.dt.month.isin([6, 7, 8]), drop=True)
CANI_JJA = CANI_roll.where(CANI_roll.time.dt.month.isin([6, 7, 8]), drop=True)
# seasonal mean
EANI_JJA_mean = EANI_JJA.groupby(EANI_JJA.time.dt.year).mean()
CANI_JJA_mean = CANI_JJA.groupby(CANI_JJA.time.dt.year).mean()
# rolling window variance
var_roll = 21
EANI_var = EANI_JJA_mean.rolling(dim={'year': var_roll}, center=True).var()
CANI_var = CANI_JJA_mean.rolling(dim={'year': var_roll}, center=True).var()
# EANI_var = EANI.rolling(time=var_roll).var()
# CANI_var = CANI.rolling(time=var_roll).var()

CvE_r = CANI_var / EANI_var

# Phase year list

In [ ]:
# EANI_s = EANI_roll.where(EANI['time.season'] == 'JJA')
# CANI_s = CANI_roll.where(CANI['time.season'] == 'JJA')

EANI_p = np.unique(EANI_JJA.where(EANI_JJA >= 1, drop=True).time.dt.year)
EANI_n = np.unique(EANI_JJA.where(EANI_JJA <= -1, drop=True).time.dt.year)
CANI_p = np.unique(CANI_JJA.where(CANI_JJA >= 1, drop=True).time.dt.year)
CANI_n = np.unique(CANI_JJA.where(CANI_JJA <= -1, drop=True).time.dt.year)

print(f'EANI positive: {EANI_p}')
print(f'EANI negative: {EANI_n}')
print(f'CANI positive: {CANI_p}')
print(f'CANI negative: {CANI_n}')

In [ ]:
# debuggin having positive and negative phase on the same year
print(('EANI positive and negatative intersection', set(EANI_p) & set(EANI_n)))
print(('CANI pos and neg intersection', set(CANI_p) & set(CANI_n)))

print(('CANI and EANI positive intersection', set(CANI_p) & set(EANI_p)))

# Plotting

## EOFs

In [ ]:
j = 0
for i in scaled_EOF['mode'].values:
    mode = scaled_EOF.sel(mode=i)
    ds_map(mode, name=f'EOF{i} scaled by PCs STD {xplained_var[j] * 100 }%')
    j += 1

## PCs

In [ ]:
plt.figure(figsize=(12, 4))
plt.plot(PC1.time, PC1, label='PC1')
plt.plot(PC2.time, PC2, label='PC2')
plt.plot(PC3.time, PC3, label='PC3')
plt.legend()
plt.grid()
plt.show()

## CANI EANI patterns

In [ ]:
ds_map(EATLs, name=f'EAN scaled pattern')
ds_map(CATLs, name=f'CAN scaled pattern')

## EANI and CANI

In [ ]:
index_plot(EANI_roll, name1='EANI', threshold=1)
index_plot(CANI_roll, name1='CANI', threshold=1)

## Variability

In [ ]:
plt.figure(figsize=(12, 4))
plt.plot(EANI_var.year, EANI_var[:], label='EANI Variance', color='blue', linestyle='--')
plt.plot(CANI_var.year, CANI_var[:], label='CANI Variance', color='orangered', linestyle='--')
plt.plot(CvE_r.year, CvE_r, label='Variance ratio C / E', color='black')
plt.axline((1970, 1), slope=0, color='gray', linestyle='--')
plt.legend()
plt.show()

# Export

In [ ]:
# time series to export
# EANI_roll
# CANI_roll
# PC1, PC2, PC3
# EANI_var
# CANI_var
# CvE_r

timeseries = xr.Dataset(
    {
        "EANI": (['time'], EANI_roll.data, {
            "description": f"Eastern Atlantic Niño Index. PCs scaled by STD and roll mean of {roll} months"
        }),
        "CANI": (['time'], CANI_roll.data, {
            "description": f"Central Atlantic Niño Index. PCs scaled by STD and roll mean of {roll} months"
        }),
        "PC1": (['time'], PC1.data, {
            "description": "PC1 from EOFa"
        }),
        "PC2": (['time'], PC2.data, {
            "description": "PC2 from EOFa"
        }),
        "PC3": (['time'], PC3.data, {
            "description": "PC3 from EOFa"
        }),
    },
    coords={"time": EANI_roll['time'].values}
)

timeseries.attrs.update({
    "title": "Atlantic Niño Index Timeseries",
    "description": f"Contains EANI, CANI, and PCs from EOFa done on HADISSTv1.1",
    "created": "2025-06-11"
})

# ~/folder does not work here
timeseries.to_netcdf("Results/CESM21_CANI_EANI_HADISST.nc")